In [1]:
#| default_exp templates

In [2]:
#| export
from __future__ import annotations

# templates

> Template file discovery and processing


<!-- # Prologue -->

In [3]:
#| export
import fnmatch
import shutil
from pathlib import Path


In [4]:
#| export
from nooprompter.config import find_config
from nooprompter.config import get_all_tags
from nooprompter.config import load_config
from nooprompter.tags import substitute_tags_in_file


In [5]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *


# File Discovery

In [6]:
#| export
def get_template_files(source_dir: str | Path, extensions: list[str]) -> list[Path]:
    "Find all template files with specified extensions"
    source_dir = Path(source_dir)
    if not source_dir.exists(): return []
    files = []
    for ext in extensions:
        if not ext.startswith('.'): ext = f'.{ext}'
        files.extend(source_dir.rglob(f'*{ext}'))
    return [f for f in files if f.is_file()]


In [7]:
# Test get_template_files
templates_dir = Path('../templates/Cursor')
if templates_dir.exists():
    files = get_template_files(templates_dir, ['.mdc'])
    test_is(len(files) > 0, True)
    test_is(all(f.suffix == '.mdc' for f in files), True)

In [8]:
#| export
def should_ignore(path: Path, ignore_patterns: list[str]) -> bool:
    "Check if path matches any ignore pattern"
    if not ignore_patterns: return False
    path_str = str(path)
    path_parts = path.parts
    for pattern in ignore_patterns:
        # Check full path match
        if fnmatch.fnmatch(path_str, pattern): return True
        # Check filename match
        if fnmatch.fnmatch(path.name, pattern): return True
        # Check if any path component matches
        if pattern in path_parts: return True
        # Check pattern match on any component
        if any(fnmatch.fnmatch(part, pattern) for part in path_parts): return True
    return False

In [9]:
test_eq(should_ignore(Path('test.pyc'), ['*.pyc']), True)
test_eq(should_ignore(Path('test.txt'), ['*.pyc']), False)
test_eq(should_ignore(Path('normal.md'), []), False)

In [10]:
test_eq(should_ignore(Path('__pycache__/test.py'), ['__pycache__']), True)

# Template Processing


In [11]:
#| export
def process_template(
    src: Path,                         # Source template file
    dst: Path,                         # Destination file
    tags: dict[str, str],              # Tag substitution dictionary
    no_substitution: list[str]|None = None, # List of file patterns to copy without substitution
    **options                          # Options for substitute_tags
) -> Path:                             # Path to destination file
    "Process a single template file"
    if no_substitution is None: no_substitution = []
    should_copy_as_is = any(
        fnmatch.fnmatch(str(src), pattern) or fnmatch.fnmatch(src.name, pattern)
        for pattern in no_substitution
    )
    dst.parent.mkdir(parents=True, exist_ok=True)
    if should_copy_as_is: shutil.copy2(src, dst)
    else: substitute_tags_in_file(src, dst, tags, **options)
    return dst

In [12]:
#| export
def process_vendor(
    config: dict,
    vendor_name: str,
    root_path: Path | None = None
) -> list[Path]:
    "Process all templates for a specific vendor"
    if root_path is None: root_path = Path.cwd()
    
    templates = config.get('templates', {})
    if vendor_name not in templates:
        raise ValueError(f"Vendor '{vendor_name}' not found in config")
    
    vendor_config = templates[vendor_name]
    
    # Convention: source is templates/{vendor_name}/
    source_dir = root_path / 'templates' / vendor_name
    
    # Special case for meta - hardcoded destination
    if vendor_name == 'meta':
        dest_dir = root_path / 'meta'
    else:
        dest = vendor_config.get('dest')
        if not dest:
            raise ValueError(f"Vendor '{vendor_name}' missing 'dest' configuration")
        dest_dir = root_path / dest
    
    # Default extensions for text files
    extensions = ['.md', '.mdc', '.txt', '.yml', '.yaml']
    
    tags = config['tags']
    general = config['general']
    ignore_patterns = general.get('ignore', [])
    
    template_files = get_template_files(source_dir, extensions)
    processed = []
    
    for src_file in template_files:
        if should_ignore(src_file, ignore_patterns):
            continue
        rel_path = src_file.relative_to(source_dir)
        dst_file = dest_dir / rel_path
        process_template(src_file, dst_file, tags)
        processed.append(dst_file)
    
    return processed

In [13]:
#| export
def process_all(
    config: dict,
    root_path: Path | None = None
) -> dict[str, list[Path]]:
    "Process all configured templates"
    if root_path is None: root_path = Path.cwd()
    
    results = {}
    templates = config.get('templates', {})
    
    for vendor_name in templates:
        try:
            files = process_vendor(config, vendor_name, root_path)
            if files:
                results[vendor_name] = files
        except Exception as e:
            print(f"Warning: Failed to process {vendor_name}: {e}")
    
    return results

In [14]:
# Test process_vendor with real config
config = load_config()
config_path = find_config()
base_path = config_path.parent if config_path else Path.cwd()

# Test meta (special case)
if (base_path / 'templates/meta').exists():
    files = process_vendor(config, 'meta', base_path)
    test_is(isinstance(files, list), True)
    # Check files were created in meta/
    if files:
        test_is(all(f.is_relative_to(base_path / 'meta') for f in files), True)

# Test Cursor vendor
if (base_path / 'templates/Cursor').exists():
    cursor_dest = base_path / config['templates']['Cursor']['dest']
    files = process_vendor(config, 'Cursor', base_path)
    test_is(isinstance(files, list), True)
    if files:
        test_is(all(f.is_relative_to(cursor_dest) for f in files), True)

In [18]:
# Test process_all
results = process_all(config, base_path)
test_is(isinstance(results, dict), True)
test_is('meta' in results or 'Cursor' in results, True)

# Each vendor should return list of paths
for vendor, files in results.items():
    test_is(isinstance(files, list), True)

In [19]:
# Test error handling
test_fail(
    lambda: process_vendor(config, 'NonExistentVendor', base_path),
    contains='not found'
)

----
<!-- # Colophon -->

In [20]:
#|hide
#|eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [22]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    nb_path = '03_templates.ipynb'
    # nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)